In [2]:
from astropy.table import Table
import numpy as np
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
#import pandas as pd
from astropy.coordinates import Galactocentric
from scipy import interpolate

In [3]:
init_para_data = 'LAMOST_HVSs_orbital_integration_initialparas.csv'

In [4]:
def cal_one_3Dloc(ra,dec,dist,frame='fk5'): # 根据赤经、赤纬、距离计算笛卡尔坐标系X Y Z
    coord = SkyCoord(ra=ra, dec=dec, distance=dist, frame=frame).transform_to(Galactocentric).cartesian
    return coord.x,coord.y,coord.z

def cal_one_3Dloc_mc(ra,dec,dist,dist_low,dist_high,n=10000): # 蒙特卡洛计算，ra,dec in deg, dist in pc, 输入不带单位
    y_err = np.random.normal(loc=0, scale=((dist_high-dist)+(dist-dist_low))*0.5, size=n) # 新的标准差是否科学？
    yy = dist + y_err # 抽样10000个距离
    
    xs,ys,zs,rs,Rs = np.empty(n),np.empty(n),np.empty(n),np.empty(n),np.empty(n)
    for i in range(n):
        if yy[i]<=0: # 距离小于0则扔掉
            continue
        x,y,z = cal_one_3Dloc(ra=ra*u.degree,dec=dec*u.degree,dist=yy[i]*u.pc,frame='fk5')
        xs[i] = x.to(u.kpc).value
        ys[i] = y.to(u.kpc).value
        zs[i] = z.to(u.kpc).value
        rs[i] = np.sqrt(xs[i]**2+ys[i]**2+zs[i]**2) # 银心距kpc
        Rs[i] = np.sqrt(xs[i]**2+ys[i]**2) # 银心距kpc


    xmid = np.percentile(xs,50)
    xl = np.percentile(xs,16)
    xh = np.percentile(xs,84)
    xerrup = xh-xmid
    xerrdn = xmid-xl
    
    ymid = np.percentile(ys,50)
    yl = np.percentile(ys,16)
    yh = np.percentile(ys,84)
    yerrup = yh-ymid
    yerrdn = ymid-yl

    zmid = np.nanpercentile(zs,50)
    zl = np.nanpercentile(zs,16)
    zh = np.nanpercentile(zs,84)
    zerrup = zh-zmid
    zerrdn = zmid-zl
    
    rmid = np.nanpercentile(rs,50)
    rl = np.nanpercentile(rs,16)
    rh = np.nanpercentile(rs,84)
    rerrup = rh-rmid
    rerrdn = rmid-rl

    Rmid = np.nanpercentile(Rs,50)
    Rl = np.nanpercentile(Rs,16)
    Rh = np.nanpercentile(Rs,84)
    Rerrup = Rh-Rmid
    Rerrdn = Rmid-Rl
    
    return (xmid,xerrup,xerrdn, ymid,yerrup,yerrdn, zmid,zerrup,zerrdn, rmid,rerrup,rerrdn, Rmid,Rerrup,Rerrdn) 
    #X,Y,Z(银心参考系,**X指向银心为正**)R,- kpc
# 测试 8kpc，太阳为-8kpc
print(1000*np.array(cal_one_3Dloc_mc(ra=298.317, dec=37.778, dist=0.0001,dist_low=0.00001,dist_high=0.00011,n=1000)))

[-8.12197334e+03  1.44017989e-05  1.37018539e-05  9.55388686e-05
  4.58566525e-05  4.76162571e-05  2.08000091e+01  4.38699694e-06
  4.55533411e-06  8.12199997e+03  1.43415608e-05  1.40979548e-05
  8.12197334e+03  1.37018521e-05  1.44018006e-05]


In [5]:
from PyAstronomy.pyasl import gal_uvw

'''
U: Velocity (km/s) positive toward the Galactic center 
V: Velocity (km/s) positive in the direction of Galactic rotation 
W: Velocity (km/s) positive toward the North Galactic Pole
'''

lsr_huang = (11.69, 10.16, 7.67) # Solar peculiar motion from Wang+2021
# print(gal_uvw(ra=16., dec=20., pmra=2., pmdec=1., distance=4000, vrad=100, lsr=lsr_huang))


def cal_one_3Dvel(ra,dec,pmra,pmdec,dist,vrad): # 不需要单位
    U,V,W = gal_uvw(ra=ra, dec=dec, pmra=pmra, pmdec=pmdec, distance=dist, vrad=vrad, lsr=lsr_huang)
    #GC Rot NP
    return U,V,W
  
def cal_one_3Dvel_mc(ra,dec,pmra,pmdec,pmraerr,pmdecerr,dist,dist_low,dist_high,vrad,vrad_err,n=10000): # 不需要单位
    y_err = np.random.normal(loc=0, scale=((dist_high-dist)+(dist-dist_low))*0.5, size=n)
    yy = dist + y_err # 抽样距离
    
    v_err = np.random.normal(loc=0, scale=vrad_err, size=n)
    vv = vrad + v_err # 抽样距离
    
    pmra_err = np.random.normal(loc=0, scale=pmraerr, size=n)
    mm = pmra + pmra_err # 抽样
    pmdec_err = np.random.normal(loc=0, scale=pmdecerr, size=n)
    nn = pmdec + pmdec_err # 抽样
    
    xs,ys,zs = np.empty(n),np.empty(n),np.empty(n)
    rs = np.empty(n)
    for i in range(n):
        if yy[i]<=0:
            continue
        x,y,z = cal_one_3Dvel(ra,dec,mm[i],nn[i],yy[i],vv[i])
        xs[i] = x
        ys[i] = y
        zs[i] = z
        rs[i] = np.sqrt(x**2+(y+230)**2+z**2) #总速度

    xmid = np.nanpercentile(xs,50)
    xl = np.nanpercentile(xs,16)
    xh = np.nanpercentile(xs,84)
    xerrup = xh-xmid
    xerrdn = xmid-xl
    
    ymid = np.nanpercentile(ys,50)
    yl = np.nanpercentile(ys,16)
    yh = np.nanpercentile(ys,84)
    yerrup = yh-ymid
    yerrdn = ymid-yl

    zmid = np.nanpercentile(zs,50)
    zl = np.nanpercentile(zs,16)
    zh = np.nanpercentile(zs,84)
    zerrup = zh-zmid
    zerrdn = zmid-zl
    
    rmid = np.nanpercentile(rs,50)
    rl = np.nanpercentile(rs,16)
    rh = np.nanpercentile(rs,84)
    rerrup = rh-rmid
    rerrdn = rmid-rl
    
    return (xmid,xerrup,xerrdn, ymid,yerrup,yerrdn, zmid,zerrup,zerrdn, rmid,rerrup,rerrdn) #kpc

# 测试 PN K5-3 from Awad+2023RAA
U,V,W=cal_one_3Dvel(ra=262.6717,dec=-23.7501,pmra=-4.363,pmdec=-9.627,dist=4540,vrad=144.9)
U,V+230,W

(np.float64(169.28277004719456),
 np.float64(22.161714738022795),
 np.float64(-11.873396729745268))

In [6]:
# 先用蒙特卡洛生成轨道积分需要的输入参数
# ras,decs,Ds,pmras,pmdecs,rvs
def generate_inputs(ra,dec, pmra,pmdec,pmraerr,pmdecerr, dist,dist_low,dist_high, vrad,vrad_err):
    n = 1
    while True:
        y_err = np.random.normal(loc=0, scale=((dist_high-dist)+(dist-dist_low))*0.5, size=n)
        yy = dist + y_err # 抽样1个距离
        if yy > 0: # 避免负的距离
            break
    v_err = np.random.normal(loc=0, scale=vrad_err, size=n)
    vv = vrad + v_err # 抽样1个视向速度
    
    pmra_err = np.random.normal(loc=0, scale=pmraerr, size=n)
    mm = pmra + pmra_err # 抽样1个pmra
    pmdec_err = np.random.normal(loc=0, scale=pmdecerr, size=n)
    nn = pmdec + pmdec_err # 抽样1个pmdec    
    return ra, dec, yy[0], mm[0], nn[0], vv[0]

# 测试HVS9
ras = 193.9120152
decs = 35.39410652
dist,dist_low,dist_high = 14160.41827, 10738.56747, 18625.78579
pmras = -7.222#proper motion in ra mas/yr
pmdecs = 1.071#proper motion in dec mas/yr
pmraerrs = 0.046
pmdecerrs = 0.056
rvs = 280.3059482#Radial velocity km/s
rverrs = 3.387654775

inputs = generate_inputs(ras,decs, pmras,pmdecs,pmraerrs,pmdecerrs, dist,dist_low,dist_high, rvs,rverrs)
inputs

(193.9120152,
 35.39410652,
 np.float64(17228.635877155713),
 np.float64(-7.190188330578666),
 np.float64(1.0473963663678643),
 np.float64(274.39481456984197))

In [7]:
# 相关参数
Ro = 8.34 *u.kpc
Vo = 230 *u.km/u.s
sm = [-11.69,10.16,7.67]*u.km/u.s #(-U,V,W) # Solar peculiar motion from Wang+2021，注意galpy的U方向默认为指向太阳

# 进行轨道积分
def start_integration_mc(ra,dec, pmra,pmdec,pmraerr,pmdecerr, dist,dist_low,dist_high, vrad,vrad_err, niter=1000, t_myr=100, ntimes=1001):
    results = []
    x0y0s = []
    for i in range(niter): # 生成不同初始参数进行蒙卡
        # 生成初始参数
        inputs = generate_inputs(ra,dec, pmra,pmdec,pmraerr,pmdecerr, dist,dist_low,dist_high, vrad,vrad_err)
        print(i, '\n(ra,dec,d_pc,pmra,pmdec,vlos) =', inputs)
        # 设置初始参数
        ras,decs,Ds,pmras,pmdecs,rvs = inputs
        vxvv = [ras*u.deg, decs*u.deg, Ds*u.pc, pmras*u.mas/u.yr, pmdecs*u.mas/u.yr, rvs*u.km/u.s]
        # 初始化轨道
        o = Orbit(vxvv=vxvv, ro=Ro, vo=Vo, solarmotion=sm, radec=True)
        # 回溯轨道
        ts = np.linspace(0,-t_myr,ntimes)*u.Myr
        o.integrate(ts, MWPotential2014) # 进行反向轨道积分
        o.plot()
        o.plot(d1='x',d2='y')
        o.plot(d1='x',d2='z')
        o.plot(d1='y',d2='z')
        # 提取轨道数据
        data = [[t_, -o.x(t_), o.y(t_), o.z(t_), -o.vx(t_), o.vy(t_), o.vz(t_)] for t_ in ts] # ***x,vx现在是以银心方向为正(20241008)
        xs,ys,zs = [-o.x(t_) for t_ in ts],[o.y(t_) for t_ in ts],[o.z(t_) for t_ in ts]
        print(xs[0],ys[0],ts[0])
        xs,ys,zs = np.array(xs),np.array(ys),np.array(zs)
        # 插值得到银盘交点
        # if max(xs)>10:
        #     fzx = interpolate.interp1d(zs[(zs<0.4)&(xs<10)], xs[(zs<0.4)&(xs<10)], bounds_error=False, fill_value=np.nan, kind='linear')
        # else:
        #     fzx = interpolate.interp1d(zs[xs<10], xs[xs<10], bounds_error=False, fill_value=np.nan, kind='linear')
        #fzx = interpolate.interp1d(zs, xs, bounds_error=False, fill_value=np.nan, kind='linear')
        #x0 = fzx(0)
        #fzy = interpolate.interp1d(zs[zs<4], ys[zs<4], bounds_error=False, fill_value=np.nan, kind='linear')
        #y0 = fzy(0)
        #x0y0s.append((x0,y0)) # 存储银盘交汇点xy坐标
        results.append(np.sqrt(o.vx(ts[0])**2+o.vy(ts[0])**2+o.vz(ts[0])**2)) # 总速度作为结果
        print(f'ini_t, end_t:  {ts[0].value}, {ts[-1].value} Myr') # 起始时间，反向终止时间
        print(f'vtot_int, vtot_end:  '+\
          f'{np.sqrt(o.vx(ts[0])**2+o.vy(ts[0])**2+o.vz(ts[0])**2):.1f}, {np.sqrt(o.vx(ts[-1])**2+o.vy(ts[-1])**2+o.vz(ts[-1])**2):.1f} km/s')
        #print(f'(x0,y0)(kpc) = {x0:.3f},{y0:.3f}')
    return results

In [8]:
# 下面绘制银盘交点分布
from scipy.stats import gaussian_kde

In [9]:
x0 = crosses[:,0] # x坐标
y0 = crosses[:,1] # 对应的y坐标

x = x0[~np.isnan(x0)]
y = y0[~np.isnan(x0)]
x = x[y>1.2]
y = y[y>1.2]
# 计算二维高斯核密度估计
xy = np.vstack([x, y])
z = gaussian_kde(xy)(xy)

# 排序z值以找到95%置信区间的边界
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]
threshold = np.percentile(z, 90)

# 创建网格并计算每个网格点上的密度
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
X, Y = np.mgrid[xmin:xmax:800j, ymin:ymax:800j]
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(gaussian_kde(xy)(positions).T, X.shape)

NameError: name 'crosses' is not defined

In [ ]:
# 绘制等高线图
plt.figure(figsize=(8, 6),dpi=100)
#plt.contourf(X, Y, Z, levels=[threshold, Z.max()], colors='none', hatches=['//'], extend='lower')
plt.contourf(X, Y, Z, levels=[threshold, Z.max()], colors='magenta', extend='lower',alpha=0.7)
plt.contour(X, Y, Z, levels=[threshold], colors='magenta', linewidths=1)
plt.scatter(x, y, c='b', s=2, edgecolor='None')
plt.scatter(0,0,c='k',marker='x',s=40)
#plt.colorbar(label='Density')
plt.xlabel('X (kpc)')
plt.ylabel('Y (kpc)')
plt.axhline(y=1.2)
#plt.xlim(15,-10)
#plt.ylim(-3,1)
plt.title('HVS8')
plt.show()
plt.close()
len(x)

In [ ]:
np.save('HVS8_cross_xy2.npy',(x,y))
np.save('HVS8_cross_kde_z2.npy',z)
np.save('HVS8_cross_grid_XYZ2.npy',(X,Y,Z))
# load = np.load('HVS9_cross_xy.npy')
# load = np.load('HVS9_cross_kde_z.npy')
# load = np.load('HVS9_cross_grid_XYZ.npy')
# load